In [ ]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [ ]:
# construct a 3d convolutional network
x = tf.placeholder(tf.float32, shape=[None, 12,12,12])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv3d(x, W):
  return tf.nn.conv3d(x, W, strides=[1, 1, 1, 1, 1], padding='SAME')

def max_pool_2x2x2(x):
  return tf.nn.max_pool3d(x, ksize=[1, 2, 2, 2, 1],
                        strides=[1, 2, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([4, 4, 4, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,12,12,12,1])

h_conv1 = tf.nn.relu(conv3d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2x2(h_conv1)

W_conv2 = weight_variable([4, 4, 4, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv3d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2x2(h_conv2)

W_fc1 = weight_variable([3 * 3 * 3 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 3*3*3*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

saver = tf.train.Saver()
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())


In [ ]:
import json
from pprint import pprint

with open('touchdata.json') as data_file:    
    data = json.load(data_file)

In [ ]:
import numpy as np
import operator
import math
from functools import reduce

# anti aliased n-dimensional line
# works well for lines <1 pixel length
def aline(s, e, cb):
    def _fpart(x):
        return x - math.floor(x)
    def _rfpart(x):
        return 1 - _fpart(x)
    
    diff = [abs(j-i) for (j, i) in zip(e, s)]
    maxd = max(diff)
    if maxd <= 0: return
    maxi = np.argmax(diff)
    sign = int(np.sign(e[maxi] - s[maxi]))
    grad = [(j-i) / maxd for (j, i) in zip(e, s)]
    sv = np.zeros_like(s)
    sv[maxi] = sign
    
    if sign < 0: (s, e) = (s + sv, e + sv) 
    sf = _rfpart(s[maxi]) if sign > 0 else _fpart(s[maxi])
    q = (s + np.multiply(grad, sf))
    sq = (s - np.multiply(sv, 1-sf))

    ef = _rfpart(e[maxi]) if sign > 0 else _fpart(e[maxi])
    qe = (e - np.multiply(grad, 1 - ef))
    eq = (e + np.multiply(sv, ef))
    
    #print(sf)
    #print(q)
    #print(qe)
    
    for n in range(2**len(s)):
        if n & (2**maxi): continue
        nd = [0 != (n & (2**a)) for a in range(len(s))]
        o = reduce(operator.mul, 
                   [_fpart(v) if b else _rfpart(v) for (v, b) in zip(sq, nd)])
        if o == 0: continue
        cb(tuple((sq + nd).astype(int)), o * sf)

    for _ in range(int(q[maxi]), int(qe[maxi] + sign), sign):
        for n in range(2**len(q)):
            if n & (2**maxi): continue
            nd = [0 != n & (2**a) for a in range(len(q))]
            o = reduce(operator.mul, 
                       [_fpart(v) if b else _rfpart(v) for (v, b) in zip(q, nd)])
            if o == 0: continue
            cb(tuple((q + nd).astype(int)), o)
        q = np.add(q, grad)

    for n in range(2**len(e)):
        if n & (2**maxi): continue
        nd = [0 != (n & (2**a)) for a in range(len(e))]
        o = -reduce(operator.mul, 
                   [_fpart(v) if b else _rfpart(v) for (v, b) in zip(qe, nd)])
        if o == 0: continue
        cb(tuple((qe + nd).astype(int)), o * ef)

In [ ]:
# n-dimensional line without anti aliasing
def line(s, e, cb):
    diff = [abs(i-j) for (i,j) in zip(s, e)]
    sign = [1 if i < j else -1 for (i,j) in zip(s, e)]
    maxd = max(diff)
    p = [maxd/2]*len(s)
    q = list(s)
    
    for _ in range(maxd):
        cb(tuple(q))
        for i in range(len(s)):
            p[i] -= diff[i]
            if (p[i] < 0):
                p[i] += maxd
                q[i] += sign[i]

In [ ]:
from collections import namedtuple
from itertools import chain

Point = namedtuple("Point", "x y t p")

def make_picture(entry):
    drawing = entry['strokes']
    #drawing2 = [[Point(**p) for p in s] for s in drawing]
    #def point_min(p1, p2):
    #    return Point(min(p1.x, p2.x), min(p1.y, p2.y), min(p1.t, p2.t), p1.p)
    #mini = reduce(point_min, chain(*drawing2), drawing2[0][0])
    
    mini = dict(drawing[0][0])
    maxi = dict(drawing[0][0])
    for s in drawing:
        for p in s:
            mini['x'] = min(mini['x'], p['x'])
            mini['y'] = min(mini['y'], p['y'])
            mini['t'] = min(mini['t'], p['t'])
            maxi['x'] = max(maxi['x'], p['x'])
            maxi['y'] = max(maxi['y'], p['y'])
            maxi['t'] = max(maxi['t'], p['t'])

    for s in drawing:
        for p in s:
            p['x'] = 11 * (p['x'] - mini['x']) / (maxi['x'] - mini['x'])
            p['y'] = 11 * (p['y'] - mini['y']) / (maxi['y'] - mini['y'])
            p['t'] = 11 * (p['t'] - mini['t']) / (maxi['t'] - mini['t'])
    
    picture = np.zeros((12, 12, 12))
    def pixel(p, a):
        try:
            picture[p] += a
        except IndexError:
            pass

    for stroke in drawing:
        for (s, e) in zip(stroke, stroke[1:]):
            #spoint = (round(s['t']), round(s['y']), round(s['x']))
            #epoint = (round(e['t']), round(e['y']), round(e['x']))
            #line(spoint, epoint, pixel)
            spoint = (s['t'], s['y'], s['x'])
            epoint = (e['t'], e['y'], e['x'])
            aline(spoint, epoint, pixel)

    return picture

In [ ]:
i = 0
np.set_printoptions(threshold=np.nan, linewidth=140, precision=4, suppress=True)
for challenge in data['submission']:
    if len(challenge) != 10:
        continue # filter out submissions we cannot train
    pictures = np.array([make_picture(c) for c in challenge])
    classification = sess.run(tf.one_hot([int(c['description']) for c in challenge], 10))
    
    #pprint(classification)
    #pprint(pictures)
    
    if i%2 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:pictures, y_: classification, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x:pictures, y_: classification, keep_prob: 0.5})
    i = i + 1